## MobileNetV2 (Image Classification, Lightweight)

In [1]:
import torch
import torchvision
from torchvision import transforms
import numpy as np
from tvm import relay


mobilenet = torchvision.models.mobilenet_v2(pretrained=True).eval()
input_shape = [1, 3, 224, 224]
x = torch.randn(input_shape)

y = mobilenet(x)
print("MobileNetV2 output shape:", y.shape)
scripted_mobilenet = torch.jit.trace(mobilenet, x).eval()

# Convert to TVM
input_name = "input0"
shape_list = [(input_name, x.shape)]
mod_mobilenet, params_mobilenet = relay.frontend.from_pytorch(scripted_mobilenet, shape_list)

print("✅ MobileNetV2 successfully converted to Relay")



/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2 output shape: torch.Size([1, 1000])
✅ MobileNetV2 successfully converted to Relay


## Compile Mobilenet

In [2]:
import tvm
from tvm.contrib import graph_executor
import os

# Choose your target architecture
# Example: x86 CPU
target_str = "llvm -mtriple=x86_64-linux-gnu -mcpu=haswell"
target = tvm.target.Target(target_str)
dev = tvm.cpu(0)

artifacts_dir = "/workspaces/TVM-Prep-guide/tvm_cpp/artifacts/MobileNet"
os.makedirs(artifacts_dir, exist_ok=True)

print(f"Compiling MobileNet...")

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod_mobilenet, target=target, params=params_mobilenet)

# Save compiled library
lib_name = os.path.join(artifacts_dir, "MobileNet_tvm.so")
lib.export_library(lib_name)

# Save graph JSON
json_path = os.path.join(artifacts_dir, "MobileNet_tvm.json")
with open(json_path, "w") as f:
    f.write(lib.get_graph_json())

# Save parameters
params_path = os.path.join(artifacts_dir, "MobileNet_tvm.params")
with open(params_path, "wb") as f:
    f.write(relay.save_param_dict(lib.get_params()))


print(f"✅ SqueezeNet compiled and exported as {lib_name}")
print(f"✅ Graph JSON saved as {json_path}")
print(f"✅ Parameters saved as {params_path}")


Compiling MobileNet...


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


✅ SqueezeNet compiled and exported as /workspaces/TVM-Prep-guide/tvm_cpp/artifacts/MobileNet/MobileNet_tvm.so
✅ Graph JSON saved as /workspaces/TVM-Prep-guide/tvm_cpp/artifacts/MobileNet/MobileNet_tvm.json
✅ Parameters saved as /workspaces/TVM-Prep-guide/tvm_cpp/artifacts/MobileNet/MobileNet_tvm.params
